# Optimize memory consumption

When setting up complex workflows, it make sense to take a look at memory consumption, especially on small device where RAM can be very limited. 
Depending on their OSs and device brand, users can track memory consumption of a code or software. Using the Task-Manager on windows, the `nvidia-smi` command for NVIDIA cards, etc. 

pyclesperanto rely on the garbage collector of python and C++ to release memory which become out of scope, hence their is no direct function from the library to call for a data release. Instead, it is possible to rely on the python `del` operation to release a variable and its data.

This notebook show an example of such memory release.

In [29]:
import numpy as np
import pyclesperanto as cle

cle.select_device(1, device_type="gpu")

(OpenCL) NVIDIA GeForce RTX 4090 (OpenCL 3.0 CUDA)
	Vendor:                      NVIDIA Corporation
	Driver Version:              535.216.01
	Device Type:                 GPU
	Compute Units:               128
	Global Memory Size:          24217 MB
	Maximum Object Size:         6054 MB
	Max Clock Frequency:         2625 MHz
	Image Support:               Yes

For overseeing memory consumption, one can use [nvidia-smi](https://nvidia.custhelp.com/app/answers/detail/a_id/3751/~/useful-nvidia-smi-queries), a command line tool that can print out how much memory is currently blocked in a given GPU, by any application:

> Two memory are return because, the system running this note book has 2 device, we will focus on the second device

In [30]:
!nvidia-smi --query-gpu=memory.used --format=csv

memory.used [MiB]
2749 MiB
787 MiB


If we then run an operation on the GPU and check memory consumption again, we should see an increase.

In [31]:
image = np.random.random((500, 1024, 1024))

blurred = cle.gaussian_blur(image)

In [32]:
!nvidia-smi --query-gpu=memory.used --format=csv

memory.used [MiB]
2753 MiB
2787 MiB


The `del` command allows to free memory. Note: The memory behind the variable may not be freed immediately, depending on how busy the system is at the moment.

In [33]:
del blurred

In [34]:
!nvidia-smi --query-gpu=memory.used --format=csv

memory.used [MiB]
2753 MiB
787 MiB


We can observe at each call of `nvidia-smi` that the memory of the second device start at `787 MB`, goes to `2787 MB` once we push our data to the device, and goes back to `787 MB` after deleting the Array wiht `del`